<a href="https://colab.research.google.com/github/igedeaguss/Magang-Pharos/blob/main/Coba_Logic_Rekomendasi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
!pip install pymssql

import pymssql
import pandas as pd
import numpy as np
import gspread
import seaborn as sns
import matplotlib.pyplot as plt
from gspread_dataframe import get_as_dataframe
import gspread
from google.colab import auth
from google.auth import default
import warnings

warnings.filterwarnings("ignore")
from google.colab import drive

#Mount Google Drive
drive.mount('/content/drive')

# Autentikasi Google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load Dataset

In [58]:
url_dokter_survey = gc.open_by_url('https://docs.google.com/spreadsheets/d/1NIOYmajDgkSY7lfKNH_wHRz0I80eeZX2OWhNtAdMw-8/edit?gid=0#gid=0')
url_database_ethical = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Qwurf3wlLDbrhWNB0XX1CgpK7GuHqCHZoAlQK_GJqek/edit?usp=sharing')
url_data_score_dokter = gc.open_by_url('https://docs.google.com/spreadsheets/d/1wIe9UOG8F6aNy1PF1UzGNQnkrcCm6DAD5zjUEnMbUo8/edit?usp=sharing')

In [60]:
#Koneksi ke Database
connection = pymssql.connect(server='35.187.243.174', user='tim_powerbi',
                             password='dGltcG93ZXJiaQ', database='mkt_insight', port = '1433')


#Menarik data kode PI dan item kode
query_1 = """
SELECT DISTINCT KodePI, [Item Kode]
FROM mkt_insight.dbo.vw_DIR10001B
WHERE KodePI IS NOT NULL
  AND KodePI <> ''
  AND (Periode LIKE '%2023%' OR Periode LIKE '%2024%')
"""


#Menarik data yang menghasilkan item kode, nama produk, zat aktif, dan total penjualan
query_2 = """
SELECT z.[Nama Produk], s.[Item Kode], z.[Zat Aktif], SUM(s.[Value Sales]) AS [Total Penjualan]
FROM mkt_insight.dbo.DIR10001B s
INNER JOIN mkt_insight.dbo.ListAllProductComplete z ON s.[Item Kode]= z.[Kode Produk PI]
GROUP BY s.[Item Kode], z.[Nama Produk], z.[Zat Aktif]"""


#Menarik data nama outlet dan kode PI
query_3 = """
SELECT DISTINCT(KodePI), NamaOutlet, Propinsi, Alamat
FROM mkt_insight.dbo.Struktur_Marketing_PI
"""

# Menarik Data PSSP
query_4 = """
SELECT * FROM mkt_insight.dbo.PSSP_PIdanPML
"""

'''
data_1 = pd.DataFrame(pd.read_sql_query(query_1, connection))
data_2 = pd.DataFrame(pd.read_sql_query(query_2, connection))'''

# Dataset untuk mencocokan nama outlet terdaftar
data_struktur_PI = pd.DataFrame(pd.read_sql_query(query_3, connection))

#Dataset produk dan zat aktif perusahaan
data_produk_zat_aktif = pd.DataFrame(pd.read_sql_query(query_2, connection))

#Dataset PSSP
data_pssp = pd.DataFrame(pd.read_sql_query(query_4, connection))

#Dataset standarisasi
data_standarisasi = pd.DataFrame(pd.read_sql_query(query_1, connection))

# Menutup koneksi
connection.close()

In [73]:
#Dataset berisi nama dokter dan kode dokter yang terdata di Database
data_dokter = get_as_dataframe(url_dokter_survey.worksheet('DB Dokter'), evaluate_formulas=True)
data_dokter = data_dokter.loc[:, ~data_dokter.columns.str.contains('^Unnamed')]

#Dataset produk pharos beserta zat aktifnya
data_produk_pharos = pd.read_excel('/content/drive/MyDrive/Dataset untuk sistem rekomendasi/Data Produk PI (Final).xlsx', dtype={'Item Kode':str})

#Dataset produk ethical yang meliputi nama produk, spesialis, dan alasan penggunaan
data_produk_ethical = get_as_dataframe(url_database_ethical.worksheet('Kategori Produk'), evaluate_formulas=True, dtype={'Kd Item': str, ' ':str})
data_produk_ethical = data_produk_ethical.loc[:, ~data_produk_ethical.columns.str.contains('^Unnamed')]

#Dataset score dokter. Menggunakan kolom Pelunasan AL
data_score_dokter =  get_as_dataframe(url_data_score_dokter.worksheet('Sheet1'), evaluate_formulas=True)
data_score_dokter = data_score_dokter.loc[:, ~data_score_dokter.columns.str.contains('^Unnamed')]

data_survey_final =  pd.read_excel('/content/drive/MyDrive/Dataset untuk sistem rekomendasi/Data Produk Survey (Final).xlsx')

# Cleaning Data

In [75]:
data_survey_zat = data_survey_final.copy()

#Cleaning data survey
data_survey_zat.iloc[:, 4:16] =data_survey_zat.iloc[:, 4:16].fillna(0)
data_survey_zat = data_survey_zat[data_survey_zat['Pack Molecule String'].isnull() == False]
data_survey_zat['Pack Molecule String'] = data_survey_zat['Pack Molecule String'].str.replace('\n', '', regex=False).str.replace(', ', ',', regex=False)
data_survey_zat['Pack Molecule String'] = data_survey_zat['Pack Molecule String'].str.replace('+', ',', regex=False)

In [76]:
data_pi_zat = data_produk_pharos.copy()

#Cleaning data pi
data_pi_zat = data_pi_zat[data_pi_zat['Pack Molecule String'].isnull() == False]
data_pi_zat['Pack Molecule String'] = data_pi_zat['Pack Molecule String'].str.replace('\n', '', regex=False).str.replace(', ', ',', regex=False)
data_pi_zat['Pack Molecule String'] = data_pi_zat['Pack Molecule String'].str.replace('+', ',', regex=False)

In [77]:
#Memfilter data yang memiliki kode outlet dan kode dokter
data_dokter = data_dokter[(data_dokter['Kd Outlet'] != '-') & (data_dokter['Kd Dokter\n(DB Pharos)'] != '-')]
data_dokter = data_dokter.drop_duplicates()

#Menggabungkan data survey dengan data dokter berdasarkan kolom 'Doctor Name'
data_merged = pd.merge(data_survey_zat, data_dokter[['Kd Outlet','Doctor Name','Kd Dokter\n(DB Pharos)','Doctor Name \n(DB Pharos)','Kd Spesialis \n(DB Pharos)','Nama Spesialis\n(DB Pharos)']], on='Doctor Name', how='left')

data_merged = data_merged[data_merged['Kd Dokter\n(DB Pharos)'].isnull() == False]

#Membuat kolom Total
data_merged["Total"] = data_merged.iloc[:, 4:16].sum(axis=1)
data_merged = data_merged[data_merged['Total'] > 0]

# Menghapus baris yang mengandung kata 'retur' di nama produk
data_merged = data_merged[~data_merged['Product Name without #'].str.contains('retur', case=False, na=False)]

#Mengubah beberapa nama kolom
data_merged = data_merged.rename(columns = {
    'Kd Outlet' : "Kode RS",
    'Kd Dokter\n(DB Pharos)' : "Kode Dokter",
    'Doctor Name \n(DB Pharos)':"Nama Dokter",
    'Hospital Name':'Nama Rumah Sakit',
    'Kd Spesialis \n(DB Pharos)' : 'Kode Spesialis',
    'Nama Spesialis\n(DB Pharos)' : 'Nama Spesialis'
})

#Melakukan groupby berdasarkan kode rs, kode dokter, zat aktif
data_zat_aktif_dokter = data_merged.groupby(['Kode RS','Kode Dokter', 'Kode Spesialis','Pack Molecule String','Dosage Form'])['Total'].sum().reset_index()
data_zat_aktif_dokter

,Kode RS,Kode Dokter,Kode Spesialis,Pack Molecule String,Dosage Form,Total
0,A1005681,A1000684,003,"AGAR,PARAFFIN OIL,PHENOLPHTHALEIN",syr,1.0
1,A1005681,A1000684,003,ATTAPULGITE,tab,18.0
2,A1005681,A1000684,003,"BACITRACIN,NEOMYCIN",pwd,1.0
3,A1005681,A1000684,003,BUPIVACAINE,inj,1.0
4,A1005681,A1000684,003,"CALCIUM,DL-LACTIC ACID,POTASSIUM,SODIUM",inf,3.0
...,...,...,...,...,...,...
76643,I1001005,Q1002060,011,METHYLPREDNISOLONE,tab,4.0
76644,I1001005,Q1002060,011,OFLOXACIN,drp,17.0
76645,I1001005,Q1002060,011,PARACETAMOL,syr,3.0
76646,I1001005,Q1002060,011,"PSEUDOEPHEDRINE,TRIPROLIDINE",tab,1547.0


In [79]:
data_pi_merged = data_pi_zat.copy()
data_pi_merged = data_pi_merged.drop(['Best Match IMS Pack','Best Similarity Score'], axis = 1)
data_pi_merged

,Item Kode,Nama Produk,Pack Molecule String,Total Penjualan,Dosage Form,Kode Form,Jenis Form
0,009100,HERCUM 0.05 % CREAM 10 G,CLOBETASOL,9.276000e+08,crm,G,TOPIC DERMATOL HAEMOR EXTERN
3,008220,IFLACORT CREAM 5 G,MOMETASONE,4.093264e+08,crm,G,TOPIC DERMATOL HAEMOR EXTERN
6,009133,DARVON 6 MG TABLET,BETAHISTINE,-2.400000e+05,tab,A,Oral Solid
7,005140,FIXIPHAR DS 30 ML,CEFIXIME,3.602060e+08,NaN,NaN,NaN
8,001906,DISFLATYL TABLET,SIMETICONE,1.555737e+10,tab,A,Oral Solid
...,...,...,...,...,...,...,...
381,008528,PLEXION 50 MG TABLET SC,EPERISONE,1.740000e+07,tab,A,Oral Solid
383,012917,QUINOBIOTIC 200 MG/100 ML INFUSION,CIPROFLOXACIN,7.063620e+08,inf,C,Parenteral
384,004810,ANTIPRESTIN 20 KAPSUL,FLUOXETINE,1.201197e+09,cap,A,Oral Solid
385,006108,TRICHOSTATIC OVULA,"METRONIDAZOLE,NYSTATIN",1.179210e+08,NaN,NaN,NaN


# Recommendation Algorithm

In [81]:
def rekomendasi_produk(data_dokter, data_produk):
    hasil_rekomendasi = []

    for _, row in data_zat_aktif_dokter.iterrows():
        kode_rs = row['Kode RS']
        kode_dokter = row['Kode Dokter']
        kode_spesialis = row['Kode Spesialis']
        zat_aktif = row['Pack Molecule String']
        total = row['Total']

        # Filter produk yang memiliki zat aktif yang sama
        produk_cocok = data_produk[data_produk['Pack Molecule String'] == zat_aktif]

        if not produk_cocok.empty:
            # Filter berdasarkan bentuk sediaan
            produk_sediaan_cocok = produk_cocok[produk_cocok['Dosage Form'] == row['Dosage Form']]

            if not produk_sediaan_cocok.empty:
                # Jika ada yang cocok berdasarkan sediaan, ambil yang pertama
                produk_terpilih = produk_sediaan_cocok.iloc[0]
                metode_pemilihan = 'Zat Aktif & Bentuk Sediaan'
            else:
                # Jika tidak ada sediaan yang cocok, ambil produk dengan total sales tertinggi
                produk_terpilih = produk_cocok.sort_values(by='Total Penjualan', ascending=False).iloc[0]
                metode_pemilihan = 'Zat Aktif & Sales Tertinggi'

            hasil_rekomendasi.append({
                'Kode RS': kode_rs,
                'Kode Dokter': kode_dokter,
                'Kode Spesialis': kode_spesialis,
                'Zat Aktif': zat_aktif,
                'Produk Rekomendasi': produk_terpilih['Nama Produk'],
                'Kode Produk': produk_terpilih['Item Kode'],
                'Metode Pemilihan': metode_pemilihan,
                'Total Peresepan': total
            })

    return pd.DataFrame(hasil_rekomendasi)

produk_rekomendasi = rekomendasi_produk(data_zat_aktif_dokter, data_pi_merged)
produk_rekomendasi

,Kode RS,Kode Dokter,Kode Spesialis,Zat Aktif,Produk Rekomendasi,Kode Produk,Metode Pemilihan,Total Peresepan
0,A1005681,A1000684,003,"BACITRACIN,NEOMYCIN",NEBACETIN POWDER 5 G,000080,Zat Aktif & Bentuk Sediaan,1.0
1,A1005681,A1000684,003,CEFADROXIL,DROVAX 500 KAPSUL,007527,Zat Aktif & Bentuk Sediaan,11.0
2,A1005681,A1000684,003,KETOROLAC,SCELTO 30 INJEKSI,007054,Zat Aktif & Bentuk Sediaan,1.0
3,A1005681,A1000684,003,KETOROLAC,SCELTO 30 INJEKSI,007054,Zat Aktif & Sales Tertinggi,10.0
4,A1005681,A1000684,003,LOPERAMIDE,RHOMUZ 2 MG FC TABLET,013038,Zat Aktif & Bentuk Sediaan,2.0
...,...,...,...,...,...,...,...,...
28585,I1001005,Q1002060,011,EPERISONE,PLEXION 50 MG TABLET SC,008528,Zat Aktif & Bentuk Sediaan,50.0
28586,I1001005,Q1002060,011,MELOXICAM,MOXAM 7.5 MG TABLET,029637,Zat Aktif & Bentuk Sediaan,80.0
28587,I1001005,Q1002060,011,METHYLPREDNISOLONE,TORAS 4 MG TABLET AG,027800,Zat Aktif & Bentuk Sediaan,4.0
28588,I1001005,Q1002060,011,PARACETAMOL,PRAXION 250MG/5ML SUSPENSI,007549,Zat Aktif & Sales Tertinggi,3.0


In [96]:
data_zat_aktif_dokter[(data_zat_aktif_dokter['Kode Dokter']=='A1000684') & (data_zat_aktif_dokter['Pack Molecule String'] == 'KETOROLAC')]

,Kode RS,Kode Dokter,Kode Spesialis,Pack Molecule String,Dosage Form,Total
11,A1005681,A1000684,003,KETOROLAC,inj,1.0
12,A1005681,A1000684,003,KETOROLAC,tab,10.0


In [91]:
data_copy_rekomendasi = produk_rekomendasi.copy()
# Manipulasi langsung di data_final untuk menggabungkan duplikat
for index, row in data_copy_rekomendasi.iterrows():
    duplikat_mask = (
        (data_copy_rekomendasi['Kode RS'] == row['Kode RS']) &
        (data_copy_rekomendasi['Kode Dokter'] == row['Kode Dokter']) &
        (data_copy_rekomendasi['Kode Produk'] == row['Kode Produk'])
    )
    matching_rows = data_copy_rekomendasi.loc[duplikat_mask]

    if len(matching_rows) > 1:
        bentuk_sediaan_mask = matching_rows['Metode Pemilihan'] == 'Zat Aktif & Bentuk Sediaan'
        if bentuk_sediaan_mask.any():
            total_sum = matching_rows['Total Peresepan'].sum()
            data_copy_rekomendasi.loc[duplikat_mask & bentuk_sediaan_mask, 'Total Peresepan'] = total_sum
            data_copy_rekomendasi = data_copy_rekomendasi.drop(matching_rows[~bentuk_sediaan_mask].index)
        else:
            total_sum = matching_rows['Total Peresepan'].sum()
            data_copy_rekomendasi.loc[duplikat_mask, 'Total Peresepan'] = total_sum
            data_copy_rekomendasi = data_copy_rekomendasi.drop(matching_rows.index[1:])

data_copy_rekomendasi

,Kode RS,Kode Dokter,Kode Spesialis,Zat Aktif,Produk Rekomendasi,Kode Produk,Metode Pemilihan,Total Peresepan
0,A1005681,A1000684,003,"BACITRACIN,NEOMYCIN",NEBACETIN POWDER 5 G,000080,Zat Aktif & Bentuk Sediaan,1.0
1,A1005681,A1000684,003,CEFADROXIL,DROVAX 500 KAPSUL,007527,Zat Aktif & Bentuk Sediaan,11.0
2,A1005681,A1000684,003,KETOROLAC,SCELTO 30 INJEKSI,007054,Zat Aktif & Bentuk Sediaan,11.0
4,A1005681,A1000684,003,LOPERAMIDE,RHOMUZ 2 MG FC TABLET,013038,Zat Aktif & Bentuk Sediaan,2.0
5,A1005681,A1000684,003,METRONIDAZOLE,BIATRON 500 FC,007440,Zat Aktif & Sales Tertinggi,2.0
...,...,...,...,...,...,...,...,...
28585,I1001005,Q1002060,011,EPERISONE,PLEXION 50 MG TABLET SC,008528,Zat Aktif & Bentuk Sediaan,50.0
28586,I1001005,Q1002060,011,MELOXICAM,MOXAM 7.5 MG TABLET,029637,Zat Aktif & Bentuk Sediaan,80.0
28587,I1001005,Q1002060,011,METHYLPREDNISOLONE,TORAS 4 MG TABLET AG,027800,Zat Aktif & Bentuk Sediaan,4.0
28588,I1001005,Q1002060,011,PARACETAMOL,PRAXION 250MG/5ML SUSPENSI,007549,Zat Aktif & Sales Tertinggi,3.0


In [93]:
produk_rekomendasi[produk_rekomendasi.duplicated(subset=['Kode Dokter', 'Kode RS', 'Kode Produk'], keep=False)]

,Kode RS,Kode Dokter,Kode Spesialis,Zat Aktif,Produk Rekomendasi,Kode Produk,Metode Pemilihan,Total Peresepan
2,A1005681,A1000684,003,KETOROLAC,SCELTO 30 INJEKSI,007054,Zat Aktif & Bentuk Sediaan,1.0
3,A1005681,A1000684,003,KETOROLAC,SCELTO 30 INJEKSI,007054,Zat Aktif & Sales Tertinggi,10.0
25,A1005681,A1000827,004,PARACETAMOL,PRAXION 250MG/5ML SUSPENSI,007549,Zat Aktif & Sales Tertinggi,36.0
26,A1005681,A1000827,004,PARACETAMOL,PRAXION 250MG/5ML SUSPENSI,007549,Zat Aktif & Sales Tertinggi,15.0
31,A1005681,A1000914,008,AZITHROMYCIN,AZTRIN 200 MG/5 ML DRY SUSPENSION,009342,Zat Aktif & Sales Tertinggi,3.0
...,...,...,...,...,...,...,...,...
28569,I1001005,I1001379,020,PARACETAMOL,PRAXION 250MG/5ML SUSPENSI,007549,Zat Aktif & Sales Tertinggi,25.0
28573,I1001005,Q1002060,011,"ALUMINIUM,DIMETICONE,MAGNESIUM",POLYSILANE SUSPENSI 100 ML,004909,Zat Aktif & Sales Tertinggi,1.0
28574,I1001005,Q1002060,011,"ALUMINIUM,DIMETICONE,MAGNESIUM",POLYSILANE SUSPENSI 100 ML,004909,Zat Aktif & Sales Tertinggi,20.0
28579,I1001005,Q1002060,011,CEFADROXIL,DROVAX 500 KAPSUL,007527,Zat Aktif & Sales Tertinggi,26.0


In [98]:
import re
'''
data_history_dokter = data_merged[['Kode RS','Kode Dokter','Nama Dokter','Pack Molecule String','Product Name without #']]
data_history_dokter = data_history_dokter.groupby(['Kode RS','Kode Dokter', 'Pack Molecule String'])['Product Name without #'].apply(
    lambda x: '; '.join(x.dropna().unique())
).reset_index()
'''
def clean_product_name(product_name):
    if pd.isna(product_name):
        return ''
    # Hapus teks dalam tanda kurung
    product_name = re.sub(r'\(.*?\)', '', product_name)
    # Hapus teks setelah tanda pemisah seperti '-', ',' yang sering digunakan untuk zat aktif
    product_name = re.split(r' - |, ', product_name)[0]
    # Hapus spasi ekstra di awal dan akhir
    return product_name.strip()

data_history_dokter = data_merged[['Kode RS', 'Kode Dokter', 'Nama Dokter', 'Pack Molecule String', 'Product Name without #']]
# Bersihkan nama produk sebelum pengelompokan
data_history_dokter['Product Name without #'] = data_history_dokter['Product Name without #'].apply(clean_product_name)

data_history_dokter = data_history_dokter.groupby(['Kode RS', 'Kode Dokter', 'Pack Molecule String'])['Product Name without #'].apply(
    lambda x: '; '.join(x.dropna().unique())
).reset_index()

In [100]:
data_final = data_copy_rekomendasi.copy()
data_final = data_final.merge(
    data_history_dokter,
    left_on = ['Kode RS','Kode Dokter','Zat Aktif'],
    right_on=['Kode RS','Kode Dokter', 'Pack Molecule String'],
    how='left'
).rename(columns={'Product Name without #': 'History Produk'})

In [101]:
data_final

,Kode RS,Kode Dokter,Kode Spesialis,Zat Aktif,Produk Rekomendasi,Kode Produk,Metode Pemilihan,Total Peresepan,Pack Molecule String,History Produk
0,A1005681,A1000684,003,"BACITRACIN,NEOMYCIN",NEBACETIN POWDER 5 G,000080,Zat Aktif & Bentuk Sediaan,1.0,"BACITRACIN,NEOMYCIN",NEBACETIN POWDER 5 G
1,A1005681,A1000684,003,CEFADROXIL,DROVAX 500 KAPSUL,007527,Zat Aktif & Bentuk Sediaan,11.0,CEFADROXIL,IH CEFADROXIL 500 MG CAPSULE; CEFADROXIL 500 M...
2,A1005681,A1000684,003,KETOROLAC,SCELTO 30 INJEKSI,007054,Zat Aktif & Bentuk Sediaan,11.0,KETOROLAC,IH KETOROLAC 10 MG TABLET; KETOROLAC 30 MG/ML ...
3,A1005681,A1000684,003,LOPERAMIDE,RHOMUZ 2 MG FC TABLET,013038,Zat Aktif & Bentuk Sediaan,2.0,LOPERAMIDE,IMODIUM 2 MG TABLET
4,A1005681,A1000684,003,METRONIDAZOLE,BIATRON 500 FC,007440,Zat Aktif & Sales Tertinggi,2.0,METRONIDAZOLE,METRONIDAZOLE 500 MG TABLET
...,...,...,...,...,...,...,...,...,...,...
24563,I1001005,Q1002060,011,EPERISONE,PLEXION 50 MG TABLET SC,008528,Zat Aktif & Bentuk Sediaan,50.0,EPERISONE,EPERISONE 50 MG TAB; MYONAL 50 MG TAB
24564,I1001005,Q1002060,011,MELOXICAM,MOXAM 7.5 MG TABLET,029637,Zat Aktif & Bentuk Sediaan,80.0,MELOXICAM,"MELOXICAM 7,5 MG TAB; MELOXICAM 7,5MG TAB; MEL..."
24565,I1001005,Q1002060,011,METHYLPREDNISOLONE,TORAS 4 MG TABLET AG,027800,Zat Aktif & Bentuk Sediaan,4.0,METHYLPREDNISOLONE,SANEXON 4 MG TAB
24566,I1001005,Q1002060,011,PARACETAMOL,PRAXION 250MG/5ML SUSPENSI,007549,Zat Aktif & Sales Tertinggi,3.0,PARACETAMOL,FASIDOL 120MG/5ML SYR; PARACETAMOL SYR 120 MG/...


In [50]:
data_merged.columns

Index([                 'Tahun',       'Nama Rumah Sakit',
                  'Doctor Name', 'Product Name without #',
                              1,                        2,
                              3,                        4,
                              5,                        6,
                              7,                        8,
                              9,                       10,
                             11,                       12,
         'Pack Molecule String',            'Dosage Form',
                      'Kode RS',            'Kode Dokter',
                  'Nama Dokter',         'Kode Spesialis',
               'Nama Spesialis',                  'Total'],
      dtype='object')

In [55]:
data_pssp.columns

Index(['Divisi KAM', 'CD Region', 'NM Region', 'NIP SM', 'NM SM', 'CD Area',
       'NM Area', 'NIP ASM', 'NM ASM', 'CD Sub Area', 'NM Sub Area', 'NIP SPV',
       'NM SPV', 'CD Group', 'NM Group', 'NIP MR', 'NM MR', 'NM Spesialis',
       'CD Cust', 'NM Cust', 'Urut', 'Divisi', 'Biaya', 'PRD Awal',
       'PRD Akhir', 'CD Product', 'NM Product', 'CD Outlet', 'NM Outlet',
       'QTY Estimasi', 'PRD Estimasi', 'Estimasi', 'PRD Pelunasan',
       'Pelunasan', 'Score Loyal'],
      dtype='object')

In [103]:
#Membuat kolom status pssp
# Buat set untuk kombinasi unik dari kode dokter, kode produk, kode rs
pssp_set = set(zip(data_pssp['CD Cust'], data_pssp['CD Product'], data_pssp['CD Outlet']))
# Tentukan nilai dengan vectorized operation
data_final['Status PSSP'] = data_final.apply(
    lambda row: 'Ya' if (row['Kode Dokter'], row['Kode Produk'], row['Kode RS']) in pssp_set else 'Tidak',
    axis=1)

,Kode RS,Kode Dokter,Kode Spesialis,Zat Aktif,Produk Rekomendasi,Kode Produk,Metode Pemilihan,Total Peresepan,Pack Molecule String,History Produk,Status PSSP
0,A1005681,A1000684,003,"BACITRACIN,NEOMYCIN",NEBACETIN POWDER 5 G,000080,Zat Aktif & Bentuk Sediaan,1.0,"BACITRACIN,NEOMYCIN",NEBACETIN POWDER 5 G,Tidak
1,A1005681,A1000684,003,CEFADROXIL,DROVAX 500 KAPSUL,007527,Zat Aktif & Bentuk Sediaan,11.0,CEFADROXIL,IH CEFADROXIL 500 MG CAPSULE; CEFADROXIL 500 M...,Tidak
2,A1005681,A1000684,003,KETOROLAC,SCELTO 30 INJEKSI,007054,Zat Aktif & Bentuk Sediaan,11.0,KETOROLAC,IH KETOROLAC 10 MG TABLET; KETOROLAC 30 MG/ML ...,Tidak
3,A1005681,A1000684,003,LOPERAMIDE,RHOMUZ 2 MG FC TABLET,013038,Zat Aktif & Bentuk Sediaan,2.0,LOPERAMIDE,IMODIUM 2 MG TABLET,Tidak
4,A1005681,A1000684,003,METRONIDAZOLE,BIATRON 500 FC,007440,Zat Aktif & Sales Tertinggi,2.0,METRONIDAZOLE,METRONIDAZOLE 500 MG TABLET,Tidak
...,...,...,...,...,...,...,...,...,...,...,...
24563,I1001005,Q1002060,011,EPERISONE,PLEXION 50 MG TABLET SC,008528,Zat Aktif & Bentuk Sediaan,50.0,EPERISONE,EPERISONE 50 MG TAB; MYONAL 50 MG TAB,Tidak
24564,I1001005,Q1002060,011,MELOXICAM,MOXAM 7.5 MG TABLET,029637,Zat Aktif & Bentuk Sediaan,80.0,MELOXICAM,"MELOXICAM 7,5 MG TAB; MELOXICAM 7,5MG TAB; MEL...",Tidak
24565,I1001005,Q1002060,011,METHYLPREDNISOLONE,TORAS 4 MG TABLET AG,027800,Zat Aktif & Bentuk Sediaan,4.0,METHYLPREDNISOLONE,SANEXON 4 MG TAB,Tidak
24566,I1001005,Q1002060,011,PARACETAMOL,PRAXION 250MG/5ML SUSPENSI,007549,Zat Aktif & Sales Tertinggi,3.0,PARACETAMOL,FASIDOL 120MG/5ML SYR; PARACETAMOL SYR 120 MG/...,Tidak


In [104]:
#Membuat kolom status sales

# Buat set untuk kombinasi unik dari 'Item Kode' dan 'Kode PI' di data_standarisasi
standarisasi_set = set(zip(data_standarisasi['Item Kode'], data_standarisasi['KodePI']))
# Tentukan nilai 'Standarisasi' dengan vectorized operation
data_final['Status Sales'] = data_final.apply(
    lambda row: 'Ya' if (row['Kode Produk'], row['Kode RS']) in standarisasi_set else 'Tidak',
    axis=1)
data_final

,Kode RS,Kode Dokter,Kode Spesialis,Zat Aktif,Produk Rekomendasi,Kode Produk,Metode Pemilihan,Total Peresepan,Pack Molecule String,History Produk,Status PSSP,Status Sales
0,A1005681,A1000684,003,"BACITRACIN,NEOMYCIN",NEBACETIN POWDER 5 G,000080,Zat Aktif & Bentuk Sediaan,1.0,"BACITRACIN,NEOMYCIN",NEBACETIN POWDER 5 G,Tidak,Ya
1,A1005681,A1000684,003,CEFADROXIL,DROVAX 500 KAPSUL,007527,Zat Aktif & Bentuk Sediaan,11.0,CEFADROXIL,IH CEFADROXIL 500 MG CAPSULE; CEFADROXIL 500 M...,Tidak,Tidak
2,A1005681,A1000684,003,KETOROLAC,SCELTO 30 INJEKSI,007054,Zat Aktif & Bentuk Sediaan,11.0,KETOROLAC,IH KETOROLAC 10 MG TABLET; KETOROLAC 30 MG/ML ...,Tidak,Ya
3,A1005681,A1000684,003,LOPERAMIDE,RHOMUZ 2 MG FC TABLET,013038,Zat Aktif & Bentuk Sediaan,2.0,LOPERAMIDE,IMODIUM 2 MG TABLET,Tidak,Tidak
4,A1005681,A1000684,003,METRONIDAZOLE,BIATRON 500 FC,007440,Zat Aktif & Sales Tertinggi,2.0,METRONIDAZOLE,METRONIDAZOLE 500 MG TABLET,Tidak,Tidak
...,...,...,...,...,...,...,...,...,...,...,...,...
24563,I1001005,Q1002060,011,EPERISONE,PLEXION 50 MG TABLET SC,008528,Zat Aktif & Bentuk Sediaan,50.0,EPERISONE,EPERISONE 50 MG TAB; MYONAL 50 MG TAB,Tidak,Tidak
24564,I1001005,Q1002060,011,MELOXICAM,MOXAM 7.5 MG TABLET,029637,Zat Aktif & Bentuk Sediaan,80.0,MELOXICAM,"MELOXICAM 7,5 MG TAB; MELOXICAM 7,5MG TAB; MEL...",Tidak,Tidak
24565,I1001005,Q1002060,011,METHYLPREDNISOLONE,TORAS 4 MG TABLET AG,027800,Zat Aktif & Bentuk Sediaan,4.0,METHYLPREDNISOLONE,SANEXON 4 MG TAB,Tidak,Tidak
24566,I1001005,Q1002060,011,PARACETAMOL,PRAXION 250MG/5ML SUSPENSI,007549,Zat Aktif & Sales Tertinggi,3.0,PARACETAMOL,FASIDOL 120MG/5ML SYR; PARACETAMOL SYR 120 MG/...,Tidak,Ya


In [105]:
data_final = data_final.drop(['Pack Molecule String'], axis=1)
data_final

,Kode RS,Kode Dokter,Kode Spesialis,Zat Aktif,Produk Rekomendasi,Kode Produk,Metode Pemilihan,Total Peresepan,History Produk,Status PSSP,Status Sales
0,A1005681,A1000684,003,"BACITRACIN,NEOMYCIN",NEBACETIN POWDER 5 G,000080,Zat Aktif & Bentuk Sediaan,1.0,NEBACETIN POWDER 5 G,Tidak,Ya
1,A1005681,A1000684,003,CEFADROXIL,DROVAX 500 KAPSUL,007527,Zat Aktif & Bentuk Sediaan,11.0,IH CEFADROXIL 500 MG CAPSULE; CEFADROXIL 500 M...,Tidak,Tidak
2,A1005681,A1000684,003,KETOROLAC,SCELTO 30 INJEKSI,007054,Zat Aktif & Bentuk Sediaan,11.0,IH KETOROLAC 10 MG TABLET; KETOROLAC 30 MG/ML ...,Tidak,Ya
3,A1005681,A1000684,003,LOPERAMIDE,RHOMUZ 2 MG FC TABLET,013038,Zat Aktif & Bentuk Sediaan,2.0,IMODIUM 2 MG TABLET,Tidak,Tidak
4,A1005681,A1000684,003,METRONIDAZOLE,BIATRON 500 FC,007440,Zat Aktif & Sales Tertinggi,2.0,METRONIDAZOLE 500 MG TABLET,Tidak,Tidak
...,...,...,...,...,...,...,...,...,...,...,...
24563,I1001005,Q1002060,011,EPERISONE,PLEXION 50 MG TABLET SC,008528,Zat Aktif & Bentuk Sediaan,50.0,EPERISONE 50 MG TAB; MYONAL 50 MG TAB,Tidak,Tidak
24564,I1001005,Q1002060,011,MELOXICAM,MOXAM 7.5 MG TABLET,029637,Zat Aktif & Bentuk Sediaan,80.0,"MELOXICAM 7,5 MG TAB; MELOXICAM 7,5MG TAB; MEL...",Tidak,Tidak
24565,I1001005,Q1002060,011,METHYLPREDNISOLONE,TORAS 4 MG TABLET AG,027800,Zat Aktif & Bentuk Sediaan,4.0,SANEXON 4 MG TAB,Tidak,Tidak
24566,I1001005,Q1002060,011,PARACETAMOL,PRAXION 250MG/5ML SUSPENSI,007549,Zat Aktif & Sales Tertinggi,3.0,FASIDOL 120MG/5ML SYR; PARACETAMOL SYR 120 MG/...,Tidak,Ya


In [38]:
data_final.to_excel('Data Final Rekomendasi.xlsx', index=False)